In [3]:
import pandas as pd
import numpy as np
import re

#READING DATA INTO PYTHON

The business.json file contains nested javascript objects, so it was necessary to convert the JSON file into a CSV file in order to process it in python.

In [1]:
'''
Convert Yelp Academic Dataset from JSON to CSV

Requires Pandas (https://pypi.python.org/pypi/pandas)

By Paul Butler, No Rights Reserved
'''
 
import json
import pandas as pd
from glob import glob
 
def convert(x):
    ''' Convert a json string to a flat python dictionary
    which can be passed into Pandas. '''
    ob = json.loads(x)
    for k, v in ob.items():
        if isinstance(v, list):
            ob[k] = ','.join(v)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                ob['%s_%s' % (k, kk)] = vv
            del ob[k]
    return ob
 
for json_filename in glob('*.json'):
    csv_filename = '%s.csv' % json_filename[:-5]
    print 'Converting %s to %s' % (json_filename, csv_filename)
    df = pd.DataFrame([convert(line) for line in file(json_filename)])
    df.to_csv(csv_filename, encoding='utf-8', index=False)

Converting business.json to business.csv


In [13]:
# business.json contains nested objects that may be empty, so read_csv
# was interpreting them as mixed types and throwing an error.
# source: http://stackoverflow.com/questions/28682562/pandas-read-csv-converting-mixed-types-columns-as-string

import warnings

target_type = str  # The desired output type

with warnings.catch_warnings(record=True) as ws:
    warnings.simplefilter("always")

    business = pd.read_csv('business.csv')
    print("Warnings raised:", ws)
    # We have an error on specific columns, try and load them as string
    for w in ws:
        s = str(w.message)
        print("Warning message:", s)
        match = re.search(r"Columns \(([0-9,]+)\) have mixed types\.", s)
        if match:
            columns = match.group(1).split(',') # Get columns as a list
            columns = [int(c) for c in columns]
            print("Applying %s dtype to columns:" % target_type, columns)
            mydata.iloc[:,columns] = mydata.iloc[:,columns].astype(target_type)

('Warnings raised:', [])


#EXAMINING THE DATA
Yelp provided a dataset with 61,184 business entries. First, let's take a look at the data provided for each business.

In [15]:
business.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61184 entries, 0 to 61183
Data columns (total 58 columns):
attributes_Accepts Credit Cards         45072 non-null object
attributes_Accepts Insurance            458 non-null object
attributes_Ages Allowed                 235 non-null object
attributes_Alcohol                      20457 non-null object
attributes_Ambience                     18547 non-null object
attributes_Attire                       20390 non-null object
attributes_BYOB                         866 non-null object
attributes_BYOB/Corkage                 1315 non-null object
attributes_By Appointment Only          7922 non-null object
attributes_Caters                       13116 non-null object
attributes_Coat Check                   3699 non-null object
attributes_Corkage                      655 non-null object
attributes_Delivery                     19924 non-null object
attributes_Dietary Restrictions         175 non-null object
attributes_Dogs Allowed             

First, we want to learn about the different types of businesses available by looking at their categories.

In [66]:
categories = business['categories'].dropna().values
unique_biz, counts = np.unique(categories, return_counts=True)
print "Unique business categories: " + str(len(unique))
print unique_biz

Unique business categories: 8046
['Accessories,Fashion,Shopping,Cosmetics & Beauty Supply,Beauty & Spas'
 'Accessories,Fashion,Shopping,Shoe Stores,Jewelry'
 'Accountants,Professional Services' ...,
 'Yoga,Physical Therapy,Health & Medical,Trainers,Fitness & Instruction,Active Life'
 'Yoga,Physical Therapy,Massage Therapy,Health & Medical,Fitness & Instruction,Active Life'
 'Yoga,Tai Chi,Adult Education,Fitness & Instruction,Active Life,Education']


As seen above, a business can claim multiple categories, so first we must determine what categories are available to examine by filtering out the duplicates. In addition, we sorted by descending count for the category to see what popular categories we can analyze.

In [69]:
# create a string containing all of the categories from each business
all_categories = ','.join(x for x in unique_biz)

#remove duplicates
unique_cat, counts = np.unique([x.strip() for x in all_categories.split(',')], return_counts=True)
for cat, count in sorted(zip(unique_cat, counts), key=lambda x: x[1], reverse=True):
    print cat + ' ' + str(count)

Restaurants 3374
Food 1555
Shopping 1450
Nightlife 1120
Bars 923
Active Life 635
Beauty & Spas 548
Event Planning & Services 538
Arts & Entertainment 535
Health & Medical 522
Home Services 450
Fashion 433
Local Services 351
American (New) 340
Sandwiches 334
Specialty Food 332
American (Traditional) 323
Coffee & Tea 322
Automotive 290
Breakfast & Brunch 280
Home & Garden 265
Hotels & Travel 242
Cafes 236
Fitness & Instruction 226
Pizza 221
Italian 215
Bakeries 211
Doctors 210
Professional Services 199
Burgers 197
Education 193
Hair Salons 191
Fast Food 172
Desserts 170
Pubs 169
Mediterranean 164
Chinese 162
Grocery 162
Lounges 158
Arts & Crafts 157
Asian Fusion 156
Delis 153
Seafood 146
Mexican 145
Sports Bars 142
Sporting Goods 138
Music Venues 136
Beer 133
Books 133
Mags 133
Music & Video 133
Wine & Spirits 133
Flowers & Gifts 129
Hair Removal 129
Skin Care 128
Vegetarian 127
Venues & Event Spaces 127
Steakhouses 126
Ice Cream & Frozen Yogurt 122
Women's Clothing 122
Barbeque 121
Cate

Similarly, we can do the same for cities to determine which has the most number of Yelp reviews.

In [68]:
cities = business['city'].values
unique, counts = np.unique(cities, return_counts=True)
print 'Uniqustr(len(unique))
for city, count in sorted(zip(unique, counts), key=lambda x: x[1], reverse=True):
    print str(city) + ' ' + str(count)

378
Las Vegas 13601
Phoenix 8410
Charlotte 4224
Scottsdale 4039
Edinburgh 3031
Pittsburgh 2724
Mesa 2347
Tempe 2258
Henderson 2130
Montréal 1870
Chandler 1867
Madison 1758
Montreal 1384
Glendale 1377
Gilbert 1263
Karlsruhe 806
Peoria 688
North Las Vegas 616
Surprise 448
Champaign 398
Goodyear 354
Avondale 299
Matthews 272
Queen Creek 236
Urbana 213
Waterloo 200
Cave Creek 180
Fort Mill 166
Middleton 166
Pineville 137
Kitchener 134
Fountain Hills 129
Concord 120
Laval 120
Apache Junction 116
Casa Grande 114
Maricopa 111
Fitchburg 109
Paradise Valley 93
Buckeye 92
Litchfield Park 86
Sun Prairie 85
Sun City 84
Anthem 83
Verdun 83
Homestead 71
Monona 71
Laveen 67
Tolleson 61
Wickenburg 58
Belmont 54
Ettlingen 51
San Tan Valley 47
Saint-Laurent 45
Carefree 44
Carnegie 44
Brossard 42
El Mirage 41
Verona 41
Indian Trail 39
Westmount 39
Pointe-Claire 36
Mint Hill 35
Dollard-Des-Ormeaux 29
Dollard-des-Ormeaux 29
Boulder City 28
Gold Canyon 28
Huntersville 27
Dorval 26
Florence 26
Outremont 26
W

In [40]:
idx = business.categories.str.contains(r".*[Pp]izza.*", regex=True, na=False)
biz_pizza = business[idx]

In [46]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("theme/custom.css", "r").read()
    return HTML(styles)
css_styling()